# Valuing European option using the Heston model

$$ dS_t = \mu S_t dt + \sqrt{V_t} S_t dW^1_t $$ <br>
$$ dV_t = \kappa (\theta - V_t) + \sigma \sqrt{V_t} d W^2_t $$

* $S_t$ is the asset’s value at time t
* $\mu$ is the expected growth rate of the log normal stock value
* $V_t$ is the variance of the asset $S_t$
* $W_t^1$ is the stochastic process governing the $S_t$ process
* θ is the value of mean reversion for the variance $V_t$
* $\kappa$ is the strength of mean reversion
* $\sigma$ is the volatility of volatility
* $W_t^2$ is the stochastic process governing the $V_t$ process
* The correlation between $W_t^1$ and $W_t^2$ is $\rho$

**In contrast, the Black-Scholes-Merton process assumes that the volatility is constant.**

In [1]:
import QuantLib as ql 
import numpy as np 

**Example**
* European Call option
* strike price \$130
* spot price \$127.62
* volatility of stock is 20%
* dividend yield is 1.63%
* short-term risk free rate is 0.1%
* today is 8th May, 2015
* expire on 15th Jan, 2016

In [2]:
# set evaluation date

eval_date = ql.Date(8, ql.May, 2015)
ql.Settings.instance().evaluationDate = eval_date

In [3]:
# define option data

maturity_date = ql.Date(15, ql.January, 2016)
spot_price = 127.62
strike_price = 130
volatility = 0.2
dividend_rate = 0.0163
option_type = ql.Option.Call
risk_free_rate = 0.001
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()

In [4]:
# construct European option

payoff = ql.PlainVanillaPayoff(option_type, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)

In [5]:
# construct Heston process

v0 = volatility**2 # spot variance
kappa = 0.1        # mean reversion strength
theta = v0         # mean reversion variance
sigma = 0.1        # volatility of volatility
rho = -0.75        # correlation between the asset price and its variance

spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))

flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(eval_date,
                                                     risk_free_rate,
                                                     day_count))

dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(eval_date,
                                                            dividend_rate,
                                                            day_count))

heston_process = ql.HestonProcess(flat_ts, 
                                  dividend_yield,
                                  spot_handle,
                                  v0,
                                  kappa, 
                                  theta,
                                  sigma,
                                  rho)

In [6]:
engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process))
european_option.setPricingEngine(engine)

In [7]:
european_option.NPV()

6.533866879681